<a href="https://colab.research.google.com/github/HueyVault/study_NLPs/blob/main/quests/DeepLearnings/linearregression_house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 수집


In [85]:
import os
# 환경 변수에 따라 파일 경로 설정
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None:
    # Kaggle 환경
    train_file_path =  f'/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
    test_file_path = f'/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
    print("kaggle")
else:
    # Docker 환경
    train_file_path = "../../datasets/train.csv"
    test_file_path = "../../datasets/test.csv"
    print("another")
    

kaggle


In [86]:
import pandas as pd

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [87]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [88]:
target_column = 'SalePrice'
feature_columns = ['BsmtUnfSF','LowQualFinSF','HalfBath','Fireplaces']

select_df = train_df[feature_columns]
select_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   BsmtUnfSF     1460 non-null   int64
 1   LowQualFinSF  1460 non-null   int64
 2   HalfBath      1460 non-null   int64
 3   Fireplaces    1460 non-null   int64
dtypes: int64(4)
memory usage: 45.8 KB



## 데이터 전처리
- 데이터 분석

In [89]:
from sklearn.preprocessing import StandardScaler


In [90]:
standardscaler_feature = StandardScaler()
standardscaler_feature.fit(select_df)

StandardScaler()

In [91]:
scailing_array_feature = standardscaler_feature.transform(select_df)

In [92]:
scailing_array_feature.shape, type(scailing_array_feature)

((1460, 4), numpy.ndarray)

In [93]:
y_train = train_df[target_column].values.reshape(-1, 1)  # 2차원 배열로 변환

y_train.shape

(1460, 1)

In [94]:
standardscaler_label = StandardScaler()
standardscaler_label.fit(y_train)

StandardScaler()

In [95]:
scailing_array_label = standardscaler_label.transform(y_train)

In [96]:
scailing_array_feature[5], scailing_array_label[5]

(array([-1.13928601, -0.12024172,  1.22758538, -0.95122649]),
 array([-0.47750496]))

## 데이터 분할
- train, test, validation

In [97]:
from sklearn.model_selection import train_test_split


In [98]:
train_features, test_features, train_label, test_label =\
      train_test_split(scailing_array_feature, scailing_array_label, test_size=0.2, random_state=42)
train_features.shape, test_features.shape, train_label.shape, test_label.shape

((1168, 4), (292, 4), (1168, 1), (292, 1))

In [99]:
import torch

In [100]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_label_tensor = torch.tensor(train_label, dtype=torch.float32).view(-1,1) # 행만 있는 경우 행열로 변환 해줌
# test_features_tensor = torch.tensor(test_features)
# test_label_tensor = torch.tensor(test_label)
type(train_features_tensor), type(train_label_tensor)

(torch.Tensor, torch.Tensor)

In [101]:
train_features_tensor.shape, train_label_tensor.shape

(torch.Size([1168, 4]), torch.Size([1168, 1]))

## 모델 학습



In [102]:
# Neural Network model linear regression
# model, loss function, optimizer function
class LinearRegressionNNM(torch.nn.Module) :
    def __init__(self, input_dim): # input : feature 의 열 갯수
        super(LinearRegressionNNM, self).__init__()
        # super(self).__init__()
        self.hidden_1 = torch.nn.Linear(input_dim, 64) # input_dim : feature 수 10개, output_dim : 출력 수 64 개
        self.hidden_2 = torch.nn.Linear(64, 32) # input_dim : 이전 layer output_dim 수 64개, output_dim : 출력 수 32 개
        self.hidden_3 = torch.nn.Linear(32, 16) # input_dim : 이전 layer output_dim 수 32, output_dim : 출력 수 16 개
        self.output = torch.nn.Linear(16, 1) # input_dim : 이전 layer output_dim 수 16, output_dim : 출력 수 1 개 Linear 니까
        self.relu = torch.nn.ReLU() # activation func ReLU 쓸거임. 

    def forward(self, x):
        x = self.relu(self.hidden_1(x)) # hidden layer 1 10,64
        x = self.relu(self.hidden_2(x)) # hidden layer 2 64,32
        x = self.relu(self.hidden_3(x)) # hidden layer 3 32,16
        out = self.output(x) # hidden layer 4 16,1 result 
        
        return out


In [103]:
train_features_tensor.shape[1], train_label_tensor.shape[1]

(4, 1)

In [104]:
model = LinearRegressionNNM(train_features_tensor.shape[1])

In [105]:
model

LinearRegressionNNM(
  (hidden_1): Linear(in_features=4, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [106]:
criterion = torch.nn.MSELoss() # Loss function 거의 label 타입에 따라 결정됨
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # Optimizer fuction
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Optimizer fuction

In [107]:
## 반복 학습
# loss 가 한자리 까지 줄어야 해
# for epoch in range(10):
for epoch in range(1000):
# for epoch in range(100000):
    pred_y = model.forward(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"epoch : {epoch}, loss : {loss.item()}")
    # print(f"epoch : {epoch}, loss : {loss.item()}")
    



epoch : 0, loss : 0.969213604927063
epoch : 100, loss : 0.6101902723312378
epoch : 200, loss : 0.5788611173629761
epoch : 300, loss : 0.5729031562805176
epoch : 400, loss : 0.567480206489563
epoch : 500, loss : 0.5619168281555176
epoch : 600, loss : 0.5568650960922241
epoch : 700, loss : 0.5528611540794373
epoch : 800, loss : 0.5488559603691101
epoch : 900, loss : 0.5453412532806396


## 모델 평가



In [108]:
model.eval()

LinearRegressionNNM(
  (hidden_1): Linear(in_features=4, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [109]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor) # 예측도 간은 loss function
    print(f"loss : {loss.item()}")


loss : 0.5424878001213074


In [110]:
# 결과 값(예측값, 원래값)
pred_y[0], train_label[0]

(tensor([-0.5454]), array([-0.45232089]))

## 모델 배포

In [111]:
select_test_df = test_df[feature_columns]
select_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BsmtUnfSF     1458 non-null   float64
 1   LowQualFinSF  1459 non-null   int64  
 2   HalfBath      1459 non-null   int64  
 3   Fireplaces    1459 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 45.7 KB


In [112]:
scailing_array_test = standardscaler_feature.transform(select_test_df)


In [113]:
test_features_tensor = torch.tensor(scailing_array_test, dtype=torch.float32)


In [114]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = model(test_features_tensor)


In [115]:
# inverse transform 및 반올림
pred_y_fin = np.round(standardscaler_label.inverse_transform(pred_y.reshape(-1, 1))).astype(int)

# 1차원 배열로 변환 (필요한 경우)
pred_y_fin = pred_y_fin.ravel()  # or pred_y_fin.flatten()

<ipython-input-115-f3af72695153>:2: RuntimeWarning: invalid value encountered in cast
  pred_y_fin = np.round(standardscaler_label.inverse_transform(pred_y.reshape(-1, 1))).astype(int)


In [116]:
output = pd.DataFrame({'Id': test_df.Id, 'SalePrice': pred_y_fin})
output.to_csv('submission.csv', index=False)